I see from https://roundtable.lsst.codes/rubintv/summit/auxtel/historical/2023-08-15 that  sequence numbers 68-79,  `science_program` set to `BLOCK-85` contain the data. 

Either obtain the seqNums from the EFD, or from the butler registry. 

In [1]:
from astropy.time import Time, TimeDelta
import pandas as pd
from lsst_efd_client import EfdClient
from astropy.io import fits
from astropy.visualization import ZScaleInterval
import matplotlib.pyplot as plt
import numpy as np 
from lsst.daf import butler as dafButler
from lsst.pipe.tasks.quickFrameMeasurement import QuickFrameMeasurementTask


INFO:numexpr.utils:Note: detected 128 virtual cores but NumExpr set to maximum of 64, check "NUMEXPR_MAX_THREADS" environment variable.
INFO:numexpr.utils:Note: NumExpr detected 128 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 8.
INFO:numexpr.utils:NumExpr defaulting to 8 threads.


In [2]:
efd_client = EfdClient('usdf_efd')

In [5]:
day_obs = '20230815'
butler = dafButler.Butler('/sdf/data/rubin/repo/embargo/')
datasetRefs = butler.registry.queryDatasets('raw',collections='LATISS/raw/all',
                              where=f"instrument='LATISS' AND exposure.day_obs = {day_obs}").expanded()

INFO:botocore.credentials:Found credentials in shared credentials file: /sdf/home/s/scichris/.lsst/aws-credentials.ini


In [29]:
spans = []
intra_times = {}
extra_times = {}
for i, ref in enumerate(datasetRefs):
    record = ref.dataId.records["exposure"]
    if record.science_program =='BLOCK-85':
        print(i,record.dataId['exposure'])
        spans.append(record.timespan)
        if record.observation_reason.startswith('intra'):
            intra_times[record.seq_num] = record.timespan
            
        elif record.observation_reason.startswith('extra'):
            extra_times[record.seq_num] = record.timespan

64 2023081500068
65 2023081500071
66 2023081500074
67 2023081500077
81 2023081500070
82 2023081500073
83 2023081500076
84 2023081500079
96 2023081500069
97 2023081500072
98 2023081500075
99 2023081500078


In [30]:
intra_times

{68: Timespan(begin=astropy.time.Time('2023-08-16 00:02:47.963000', scale='tai', format='iso'), end=astropy.time.Time('2023-08-16 00:03:18.207000', scale='tai', format='iso')),
 74: Timespan(begin=astropy.time.Time('2023-08-16 00:07:34.148009', scale='tai', format='iso'), end=astropy.time.Time('2023-08-16 00:08:04.387000', scale='tai', format='iso')),
 70: Timespan(begin=astropy.time.Time('2023-08-16 00:04:31.292996', scale='tai', format='iso'), end=astropy.time.Time('2023-08-16 00:05:01.539000', scale='tai', format='iso')),
 76: Timespan(begin=astropy.time.Time('2023-08-16 00:09:04.639009', scale='tai', format='iso'), end=astropy.time.Time('2023-08-16 00:09:34.881000', scale='tai', format='iso')),
 72: Timespan(begin=astropy.time.Time('2023-08-16 00:06:02.926014', scale='tai', format='iso'), end=astropy.time.Time('2023-08-16 00:06:33.162000', scale='tai', format='iso')),
 78: Timespan(begin=astropy.time.Time('2023-08-16 00:10:35.232000', scale='tai', format='iso'), end=astropy.time.Ti

The only problem with these times is that they're in TAI - pull the end readout from the `end_readout`, as then it's in UTC. 

Get the minimum and maximum obs time for the EFD query. This will yield the CWFS results as the algorithm was run :

In [18]:
t1 = spans[0].begin.utc # need to use UTC scale 
t2 = spans[-1].end.utc

In [31]:
end_readout = await efd_client.select_time_series("lsst.sal.ATCamera.logevent_endReadout", 
                                          '*', t1, t2)

I use the fact that the images are all ordered as intra/extra pairs. So the i-th is intra, and i+1 is extra. Print the `additionalValues` for these to confuirm that : 

In [101]:
end_readout

,additionalKeys,additionalValues,imageController,imageDate,imageIndex,imageName,imageNumber,imageSource,imagesInSequence,private_efdStamp,private_identity,private_kafkaStamp,private_origin,private_rcvStamp,private_revCode,private_seqNum,private_sndStamp,requestedExposureTime,timestampAcquisitionStart,timestampEndOfReadout
2023-08-16 00:02:43.611861+00:00,imageType:groupId:testType:stutterRows:stutter...,CWFS:2023-08-16T00\:00\:07.273:CWFS:0:0:0.0:IN...,O,20230815,1,AT_O_20230815_000068,68,AT,1,1.692144e+09,ATCamera,1.692144e+09,37488036,1.692144e+09,bf54d117,6266,1.692144e+09,30,1.692142e+09,1.692144e+09
2023-08-16 00:03:21.077946+00:00,imageType:groupId:testType:stutterRows:stutter...,CWFS:2023-08-16T00\:00\:07.273:CWFS:0:0:0.0:EX...,O,20230815,1,AT_O_20230815_000069,69,AT,1,1.692144e+09,ATCamera,1.692144e+09,37488036,1.692144e+09,bf54d117,6267,1.692144e+09,30,1.692144e+09,1.692144e+09
2023-08-16 00:04:26.945414+00:00,imageType:groupId:testType:stutterRows:stutter...,CWFS:2023-08-16T00\:04\:04.999:CWFS:0:0:0.0:IN...,O,20230815,1,AT_O_20230815_000070,70,AT,1,1.692144e+09,ATCamera,1.692144e+09,37488036,1.692144e+09,bf54d117,6268,1.692144e+09,30,1.692144e+09,1.692144e+09
2023-08-16 00:05:04.501528+00:00,imageType:groupId:testType:stutterRows:stutter...,CWFS:2023-08-16T00\:04\:04.999:CWFS:0:0:0.0:EX...,O,20230815,1,AT_O_20230815_000071,71,AT,1,1.692144e+09,ATCamera,1.692144e+09,37488036,1.692144e+09,bf54d117,6269,1.692144e+09,30,1.692144e+09,1.692144e+09
2023-08-16 00:05:58.567724+00:00,imageType:groupId:testType:stutterRows:stutter...,CWFS:2023-08-16T00\:05\:49.451:CWFS:0:0:0.0:IN...,O,20230815,1,AT_O_20230815_000072,72,AT,1,1.692144e+09,ATCamera,1.692144e+09,37488036,1.692144e+09,bf54d117,6270,1.692144e+09,30,1.692144e+09,1.692144e+09
2023-08-16 00:06:36.818443+00:00,imageType:groupId:testType:stutterRows:stutter...,CWFS:2023-08-16T00\:05\:49.451:CWFS:0:0:0.0:EX...,O,20230815,1,AT_O_20230815_000073,73,AT,1,1.692144e+09,ATCamera,1.692144e+09,37488036,1.692144e+09,bf54d117,6271,1.692144e+09,30,1.692144e+09,1.692144e+09
2023-08-16 00:07:29.793988+00:00,imageType:groupId:testType:stutterRows:stutter...,CWFS:2023-08-16T00\:07\:20.951:CWFS:0:0:0.0:IN...,O,20230815,1,AT_O_20230815_000074,74,AT,1,1.692144e+09,ATCamera,1.692144e+09,37488036,1.692144e+09,bf54d117,6272,1.692144e+09,30,1.692144e+09,1.692144e+09
2023-08-16 00:08:07.490279+00:00,imageType:groupId:testType:stutterRows:stutter...,CWFS:2023-08-16T00\:07\:20.951:CWFS:0:0:0.0:EX...,O,20230815,1,AT_O_20230815_000075,75,AT,1,1.692144e+09,ATCamera,1.692145e+09,37488036,1.692145e+09,bf54d117,6273,1.692145e+09,30,1.692144e+09,1.692145e+09
2023-08-16 00:09:00.286178+00:00,imageType:groupId:testType:stutterRows:stutter...,CWFS:2023-08-16T00\:08\:51.511:CWFS:0:0:0.0:IN...,O,20230815,1,AT_O_20230815_000076,76,AT,1,1.692145e+09,ATCamera,1.692145e+09,37488036,1.692145e+09,bf54d117,6274,1.692145e+09,30,1.692145e+09,1.692145e+09
2023-08-16 00:09:37.957885+00:00,imageType:groupId:testType:stutterRows:stutter...,CWFS:2023-08-16T00\:08\:51.511:CWFS:0:0:0.0:EX...,O,20230815,1,AT_O_20230815_000077,77,AT,1,1.692145e+09,ATCamera,1.692145e+09,37488036,1.692145e+09,bf54d117,6275,1.692145e+09,30,1.692145e+09,1.692145e+09


Find out the end of the readout time for each image in the intra/extra pair: 

In [104]:
intra_readout_times = []
extra_readout_times = []
intra_seqNum = []
extra_seqNum = []
i =0 
while i < len(end_readout)-1:
#for i in range(len(end_readout)-1):
    intra_readout_times.append(end_readout.index[i])
    extra_readout_times.append(end_readout.index[i+1])
    intra_seqNum.append(end_readout['imageNumber'][i])
    extra_seqNum.append(end_readout['imageNumber'][i+1])
    print('\n',end_readout['imageNumber'][i], end_readout['additionalValues'][i].split(':')[-2],
          '&',end_readout['imageNumber'][i+1], end_readout['additionalValues'][i+1].split(':')[-2])
    print(i)
    i += 2


 68 INTRA_SITCOM-778 & 69 EXTRA_SITCOM-778
0

 70 INTRA_SITCOM-778 & 71 EXTRA_SITCOM-778
2

 72 INTRA_SITCOM-778 & 73 EXTRA_SITCOM-778
4

 74 INTRA_SITCOM-778 & 75 EXTRA_SITCOM-778
6

 76 INTRA_SITCOM-778 & 77 EXTRA_SITCOM-778
8


In [107]:
# Populate the data structure from the pairs found above
for i in range(len(intra_readout_times)):
    
    # Determine time stamps for searching for metadata
    # include ability to correct for TAI if required, but set to zero for the moment

    t1 = Time(intra_readout_times[i]) - TimeDelta(30, format='sec') 
    
    t2 = Time(extra_readout_times[i]) - TimeDelta(2., format='sec')

    azel = await efd_client.select_time_series("lsst.sal.ATMCS.mount_AzEl_Encoders", 
                                               ["elevationCalculatedAngle99", "azimuthCalculatedAngle99"], 
                                               t1, t2,
                                              convert_influx_index=False)
    
    rotator = await efd_client.select_time_series("lsst.sal.ATMCS.mount_Nasmyth_Encoders",
                                                  ["nasmyth2CalculatedAngle99"], t1, t2,
                                                 convert_influx_index=False)


    
    rot_pos = np.mean(rotator['nasmyth2CalculatedAngle99'])
    el = np.mean(azel['elevationCalculatedAngle99'])
    az = np.mean(azel['azimuthCalculatedAngle99'])
    print(intra_seqNum[i], extra_seqNum[i], rot_pos, el, az)
   

68 69 0.23884503136774024 80.38136689319485 91.32066252445243
70 71 -14.447712783571854 80.75628426646466 91.16361254738193
72 73 -14.158836965996835 81.08826375524927 91.01247917264801
74 75 -13.869004070542525 81.4165591418994 90.85894232866606
76 77 -13.576174265535128 81.74319818455842 90.70180088816672
